In [1]:
from source import Dataset
from source import tools
import awkward as ak
import glob
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from keras import layers

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2023-12-04 15:10:10.978491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 15:10:12.234624: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-04 15:10:12.234653: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-04 15:10:23.648802: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
#selecting features 
branches_to_train = [
"cscRechitClusterX",
"cscRechitClusterY",
"cscRechitClusterZ",
"cscRechitClusterTimeWeighted",
"cscRechitClusterTimeSpreadWeightedAll",
"cscRechitClusternXY",
"cscRechitClusternZ",
"cscRechitClusterXSpread",
"cscRechitClusterYSpread",
"cscRechitClusterZSpread",
"cscRechitClusterXYSpread",
"cscRechitClusterRSpread",
"cscRechitClusterEtaPhiSpread",
"cscRechitClusterEtaSpread",
"cscRechitClusterPhiSpread",
"cscRechitClusterDeltaRSpread",
"cscRechitClusterMajorAxis",
"cscRechitClusterMinorAxis",
"cscRechitClusterSkewX",
"cscRechitClusterSkewY",
"cscRechitClusterSkewZ",
"cscRechitClusterKurtX",
"cscRechitClusterKurtY",
"cscRechitClusterKurtZ",
"cscRechitClusterSize",
]

#energy key
truth_value = "cscRechitCluster_match_gLLP_e"

In [6]:
#read data
dataset_signal = Dataset.Dataset("signal")
df_signal = dataset_signal.load_df("/nfs/dust/cms/user/loewetob/ML_LLP/MDS_regression/datasets/test_df_v6.h5")

Opening file  /nfs/dust/cms/user/loewetob/ML_LLP/MDS_regression/datasets/test_df_v6.h5


In [7]:
#cut clusters which don't have LLP
olen = len(df_signal)
df_signal = df_signal[ df_signal["cscRechitCluster_match_gLLP"]>=1 ]
print('cut out ' + str(olen - len(df_signal)) + ' clusters, which leaves ' + str(len(df_signal)) + ' clusters containing a LLP decay for training')

cut out 10236 clusters, which leaves 79780 clusters containing a LLP decay for training


In [15]:
# Splitting the data into train, val, test
train_data, testval_data = train_test_split(df_signal, train_size=0.6)
test_data, val_data = train_test_split(testval_data, test_size=0.5)

# assigning labels (true energy values)
train_labels = np.array(train_data[truth_value])#([1.]*len(train_data))
test_labels = np.array(test_data[truth_value])#([1.]*len(test_data))
val_labels = np.array(val_data[truth_value])#([1.]*len(val_data))

# select only branches we want
train_data = train_data[branches_to_train]
test_data = test_data[branches_to_train]
val_data = val_data[branches_to_train]

# check the shape
print(" signal training features", train_data.shape,
    "\n signal training labels", train_labels.shape,
    "\n signal test features", test_data.shape,
    "\n ignal test labels", test_labels.shape,
    "\n signal val features", val_data.shape,
    "\n signal val labels", val_labels.shape)

 signal training features (47868, 25) 
 signal training labels (47868,) 
 signal test features (15956, 25) 
 ignal test labels (15956,) 
 signal val features (15956, 25) 
 signal val labels (15956,)


In [16]:
# see what training data looks like
print(train_labels)
print(train_data)

n_features = train_data.shape[1]

[ 64.69113922 153.4440155   74.65402222 ... 173.04978943 173.95536804
 100.54255676]
       cscRechitClusterX  cscRechitClusterY  cscRechitClusterZ  \
entry                                                            
2527         -479.079712         271.723511        -720.262085   
28656        -342.130493           9.394412        -711.315491   
84923        -632.940796          11.455564         693.964844   
46747          73.034073        -396.334595       -1012.336853   
71516         585.730469          78.587311         739.977295   
...                  ...                ...                ...   
64004         225.633362         253.560608         798.928467   
5328         -409.072601         478.347778         718.115479   
32256        -349.879608          -2.012101        -994.152100   
42538        -144.072220        -210.505920        -833.869873   
80627        -541.008728         280.420013        -694.311462   

       cscRechitClusterTimeWeighted  cscRechitClusterTim

In [17]:
n_features

25